<image src="https://i.imgur.com/HrRxc5o.jpeg" width=100%>

Existen tres tipos principales de segmentación en visión por computadora: **segmentación semántica**, **segmentación por instancia** y **segmentación panóptica**. La segmentación semántica clasifica cada píxel de una imagen según su categoría (por ejemplo, “árbol” o “carretera”), sin distinguir entre diferentes objetos individuales de la misma clase. La segmentación por instancia identifica objetos individuales dentro de una misma categoría, como distinguir entre varios “autos” en una escena. Finalmente, la segmentación panóptica combina ambos enfoques, asignando a cada píxel una etiqueta semántica y, cuando corresponde, una instancia única del objeto.

<image src="https://i.imgur.com/8vukGi4.jpeg" width=85%>

<image src="https://i.imgur.com/1RLt9az.jpeg" width=85%>

<image src="https://i.imgur.com/YaTrlkl.jpeg" width=85%>

# <font color='#00FFFF'>**Diferentes tipos de Segmentación 👀**</font>

### **Importamos librerías y dependencias**

In [ ]:
!pip install 'git+https://github.com/facebookresearch/segment-anything.git' --q
!pip install jupyter_bbox_widget roboflow dataclasses-json supervision==0.23.0 --q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.5 MB/s eta 0:00:00


In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from google.colab import files
import os
from PIL import Image
import torch
import cv2
import random
import json
import supervision as sv
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation

## <font color='#00FFFF'> **1. Segmentación panóptica** </font>





### **Descargamos y configuramos SAM pre-entrenado listo para usarse**

In [ ]:
#No modifiques esta sección

In [ ]:
!mkdir -p /content/weights
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P /content//weights

In [ ]:
checkpoint = os.path.join("/content", "weights", "sam_vit_h_4b8939.pth")
print(checkpoint, "; exist:", os.path.isfile(checkpoint))

/content/weights/sam_vit_h_4b8939.pth ; exist: True


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = "vit_h"

sam = sam_model_registry[None](checkpoint=checkpoint).to(device=None)

### **Sube cualquier imagen o imágenes aquí para trabajar**

In [ ]:
uploaded = files.upload()
image_paths = []

for fn in uploaded.keys():
  if not os.path.exists('uploaded_images'):
    os.makedirs('uploaded_images')

  path = os.path.join('uploaded_images', fn)
  image_paths.append(path)

  with open(path, 'wb') as f:
    f.write(uploaded[fn])

  print('User uploaded file "{name}" with length {length} bytes and saved to {path}'.format(
      name=fn, length=len(uploaded[fn]), path=path))

# Guardamos los paths de cada image en image_path, descomenta la línea de abajo para verlos
#print("Saved image paths:", image_paths)

In [ ]:
# La posición del path depende del orden de subida de las imagenes a trabajar, si deseas cambiar la imagen,
# modifica el índice de la lista al path de la imagen a usar, no olvidar que los índices empiezan hasta 0
# y van hasta n-1 donde n es el número de imágenes

image_path = image_paths[0]
src_image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(None, cv2.COLOR_BGR2RGB)
plt.imshow(image_rgb)
plt.title('Source Image')
plt.show() #Visualización de la imagen seleccionada

### **Creación de máscaras y visualización de Segmentación Panóptica**

In [ ]:
# Crear las máscaras usando SAM a partir de la imagen RGB

mask_generator = SamAutomaticMaskGenerator(None)
src_image = cv2.imread(None)
image_rgb = cv2.None(src_image, cv2.COLOR_BGR2RGB)
sam_result = mask_generator.generate(None)

In [ ]:
mask_annotator = sv.MaskAnnotator(color_lookup=sv.ColorLookup.INDEX)
detections = sv.Detections.from_sam(sam_result=None)
annotated_image = mask_annotator.annotate(scene=src_image.copy(), detections=detections)

#Visualización de las imágenes
sv.plot_images_grid(
    images=[None, None],
    grid_size=(1, 2),
    titles=['Source Image', 'Panoptic segmentation']
)

### **Visualización de máscaras de segmentación generadas por SAM**

In [ ]:
masks = [
    mask['segmentation']
    for None
    in sorted(sam_result, key=lambda x: x['area'], reverse=True)
]

num_rows = (len(masks) + 7) // 8

sv.plot_images_grid(
    images=None,
    grid_size=(8, num_rows),
    size=(16, 16)
)

## <font color='#00FFFF'> **2. Segmentación semántica** </font>

### **Descargamos Mask2Former desde HuggingFace**

In [ ]:
#No modifiques esta sección

In [ ]:
processor = AutoImageProcessor.from_pretrained("facebook/mask2former-swin-base-coco-panoptic")
model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-base-coco-panoptic")

In [ ]:
#Algunos labels del pre-entreno para buscar imágenes relacionadas a estos

config = model.config
id2label = config.id2label

for id, label in list(id2label.items())[:10]:
  print(f"ID: {id}, Label: {label}")

### **Sube cualquier imagen o imágenes aquí para trabajar**

In [ ]:
uploaded = files.upload()
image_paths = []

for fn in uploaded.keys():
  if not os.path.exists('uploaded_images'):
    os.makedirs('uploaded_images')

  path = os.path.join('uploaded_images', fn)
  image_paths.append(path)

  with open(path, 'wb') as f:
    f.write(uploaded[fn])

  print('User uploaded file "{name}" with length {length} bytes and saved to {path}'.format(
      name=fn, length=len(uploaded[fn]), path=path))

# Guardamos los paths de cada image en image_path, descomenta la línea de abajo para verlos
#print("Saved image paths:", image_paths)

In [ ]:
# La posición del path depende del orden de subida de las imagenes a trabajar, si deseas cambiar la imagen,
# modifica el índice de la lista al path de la imagen a usar, no olvidar que los índices empiezan hasta 0
# y van hasta n-1 donde n es el número de imágenes

image = image_paths[0]
src_image = Image.open(image)
plt.imshow(src_image)
plt.title('Source Image')
plt.show() #Visualización de la imagen seleccionada

### **Obtenemos las máscaras de segmentación de Mask2Former**

In [ ]:
inputs = processor(images=None, return_tensors="pt")

In [ ]:
with torch.no_grad():
  outputs = model(**None)

In [ ]:
predicted_map = processor.post_process_semantic_segmentation(None, target_sizes=[None.size[::-1]])[0]
print(predicted_map.shape)

In [ ]:
color_palette = [list(np.random.choice(range(256), size=3)) for _ in range(len(model.config.id2label))]
print(color_palette)

### **Visualización de Segmentación Semántica**

In [ ]:
seg = None # Cambio de nombre de una variable anterior
color_seg = np.zeros((seg.shape[0], seg.shape[1], 3), dtype=np.uint8)
palette = np.array(color_palette)
for label, color in enumerate(palette):
    color_seg[seg == label, :] = color
# Convert to BGR
color_seg = color_seg[..., ::-1]

img = np.array(None)
color_seg_resized = cv2.resize(color_seg, (img.shape[1], img.shape[0]))

# Imagen y máscara (NO modificar 🙏🏼)
img = img * 0.5 + color_seg_resized * 0.5
img = img.astype(np.uint8)
fig, axes = plt.subplots(1, 2, figsize=(15, 8))

#Visualización de las imágenes
axes[0].imshow(None)
axes[0].set_title('Source Image')
axes[1].imshow(None)
axes[1].set_title('Semantic Segmentation')

plt.show()

## <font color='#00FFFF'> **3. Segmentación de instancias** </font>

### **Sube cualquier imagen o imágenes aquí para trabajar**

In [ ]:
uploaded = files.upload()
image_paths = []

for fn in uploaded.keys():
  if not os.path.exists('uploaded_images'):
    os.makedirs('uploaded_images')

  path = os.path.join('uploaded_images', fn)
  image_paths.append(path)

  with open(path, 'wb') as f:
    f.write(uploaded[fn])

  print('User uploaded file "{name}" with length {length} bytes and saved to {path}'.format(
      name=fn, length=len(uploaded[fn]), path=path))

# Guardamos los paths de cada image en image_path, descomenta la línea de abajo para verlos
#print("Saved image paths:", image_paths)

In [ ]:
# La posición del path depende del orden de subida de las imagenes a trabajar, si deseas cambiar la imagen,
# modifica el índice de la lista al path de la imagen a usar, no olvidar que los índices empiezan hasta 0
# y van hasta n-1 donde n es el número de imágenes

image = image_paths[0]
src_image = Image.open(image)
plt.imshow(src_image)
plt.title('Source Image')
plt.show() #Visualización de la imagen seleccionada

### **Obtenemos las máscaras de segmentación de Mask2Former**

In [ ]:
inputs = processor(images=src_image, return_tensors="pt")

In [ ]:
with torch.no_grad():
  outputs = model(**inputs)

In [ ]:
results = processor.post_process_instance_segmentation(None, target_sizes=[None.size[::-1]], threshold=0.9)[0]
print(results.keys())

In [ ]:
#Obtención de los IDs de las máscaras para revisarlas de forma individual

segment_to_label = {segment['id']: segment['label_id'] for segment in results["segments_info"]}

def get_available_segment_ids(results):

    segmentation = results['segmentation'].numpy()
    unique_ids = np.unique(segmentation)

    available_ids = [id for id in unique_ids]

    return available_ids

available_ids = get_available_segment_ids(results)
print("Available segment IDs:", [id.item() for id in available_ids])

In [ ]:
# Visualización individual de máscaras
def get_mask(segment_id):
  if segment_id in segment_to_label:
    print("Visualizing mask for:", model.config.id2label[None[None]])

    mask = (None['segmentation'].None() == None)
    visual_mask = (mask * 255).astype(np.uint8)
    visual_mask = Image.fromarray(visual_mask)

    plt.imshow(visual_mask)
    plt.show()
  else:
    print(f"Segment ID {segment_id} not found in segment_to_label. Skipping visualization.")
    return None

get_mask(segment_id=0)

### **Visualización Segmentación de Instancias**

In [ ]:
def draw_instance_segmentation(segmentation, segments_info):

    fig, axes_arr = plt.subplots(1, 2, figsize=(10, 8)) # Create 2 subplots
    axes_img, axes_seg = axes_arr

    axes_img.imshow(None)
    axes_img.set_title('Source Image')
    axes_seg.imshow(None)
    axes_seg.set_title('Instance Segmentation')

    instance_colors = {}
    for segment in None:
        segment_id = None['id']
        if segment_id not in instance_colors:
            instance_colors[None] = [random.randint(0, 255) for _ in range(3)]

    for segment in None:
        segment_id = segment['id']
        segment_label_id = segment['label_id']
        segment_label = model.config.id2label[None]

        mask = (segmentation.numpy() == None)

        contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            axes_seg.plot(contour[:, 0, 0], contour[:, 0, 1], color=np.array(instance_colors[segment_id]) / 255, linewidth=0.5)

        x, y = np.mean(contour, axis=0)[0]
        axes_seg.text(x, y, segment_label, color='white', backgroundcolor=np.array(instance_colors[segment_id]) / 255, fontsize=8)

    plt.show(/
#Visualización de las imágenes
draw_instance_segmentation(**None)

## <font color='#00FFFF'> **Bibliografía** </font>

Piotr Skalski. (Jan 22, 2024). How to Use the Segment Anything Model (SAM). Roboflow Blog: https://blog.roboflow.com/how-to-use-segment-anything-model-sam/

Jacob Solawetz. (Apr 7, 2023). What is Segment Anything Model (SAM)? A Breakdown.. Roboflow Blog: https://blog.roboflow.com/segment-anything-breakdown/

Rogge, N. (2025). Tutorials. Github Repository: https://github.com/NielsRogge/Transformers-Tutorials

Cheng, B., Misra, I., Schwing, A. G., Kirillov, A., & Girdhar, R. (2022). Masked-attention Mask Transformer for Universal Image Segmentation. arXiv. https://arxiv.org/abs/2112.01527

Kirillov, A., Mintun, E., Ravi, N., Mao, H., Rolland, C., Gustafson, L., Xiao, T., Whitehead, S., Berg, A. C., Lo, W.-Y., Dollár, P., & Girshick, R. (2023). Segment Anything. arXiv. https://arxiv.org/abs/2304.02643